# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [60]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [61]:
# Drop unnecessary columns
new_df = df.drop(columns='timestamp')

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [62]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader()
data = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [63]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [64]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [65]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

param_grid = {'n_factors': [20,50,100],
             'reg_all': [.02, .05, .1]}

grid_search = GridSearchCV(SVD, param_grid=param_grid, n_jobs=-1)
grid_search.fit(data)

In [66]:
# print out optimal parameters for SVD after GridSearch
print(grid_search.best_score, '\n', grid_search.best_params)

{'rmse': 0.8693353588446341, 'mae': 0.667914959886682} 
 {'rmse': {'n_factors': 100, 'reg_all': 0.05}, 'mae': {'n_factors': 20, 'reg_all': 0.02}}


In [67]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name': 'pearson', 'user_based':True})
knn_basic_cv = cross_validate(knn_basic, data, n_jobs=-1) 

In [68]:
# print out the average RMSE score for the test set
np.mean(knn_basic_cv['test_rmse'])

0.9737042548650992

In [69]:
# cross validating with KNNBaseline
knn_bline = KNNBaseline(sim_options={'name': 'pearson', 'user_based': True})
knn_bline_cv = cross_validate(knn_bline, data, n_jobs=-1)

In [70]:
# print out the average score for the test set
for item in knn_bline_cv.items():
    print(item)
print('---'*15)
np.mean(knn_bline_cv['test_rmse'])

('test_rmse', array([0.88454842, 0.87821745, 0.87632482, 0.87918613, 0.86925068]))
('test_mae', array([0.67434523, 0.67142083, 0.66655495, 0.67153701, 0.66751265]))
('fit_time', (0.47597336769104004, 0.42877984046936035, 0.4371473789215088, 0.4079427719116211, 0.4031546115875244))
('test_time', (1.6706717014312744, 1.5947833061218262, 1.5694246292114258, 1.592513084411621, 1.6145267486572266))
---------------------------------------------


0.8775054982897068

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [71]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [72]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [73]:
knn_basic = KNNBasic(sim_options={'name': 'pearson', 'user_based':True})
knn_basic.fit(dataset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [74]:
knn_basic.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=1.4999999999999998, details={'actual_k': 1, 'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [82]:
def movie_rater(movie_df,num, genre=None):
    userId = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df.genres.str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input("Please rate between 1-5. Type 'n' if you've never seen it: \n")
        if rating.lower() == 'n':
            continue
        else:
            rating_one = {'userId': userId, 'movieId': movie['movieId'].values[0], 'rating':rating}
            rating_list.append(rating_one)
            num -=1
    return rating_list
                       

In [83]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Musical')

      movieId                title                         genres
7336    77931  Stingray Sam (2009)  Comedy|Musical|Sci-Fi|Western
Please rate between 1-5. Type 'n' if you've never seen it: 
n
      movieId                                title                   genres
5206     8487  Please Don't Eat the Daisies (1960)  Children|Comedy|Musical
Please rate between 1-5. Type 'n' if you've never seen it: 
n
      movieId                                              title  \
5659    27563  Happiness of the Katakuris, The (Katakuri-ke n...   

                     genres  
5659  Comedy|Horror|Musical  
Please rate between 1-5. Type 'n' if you've never seen it: 
n
      movieId                title          genres
7786    91873  Joyful Noise (2012)  Comedy|Musical
Please rate between 1-5. Type 'n' if you've never seen it: 
n
      movieId                    title                             genres
1564     2102  Steamboat Willie (1928)  Animation|Children|Comedy|Musical
Please rate between 1

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [84]:
user_rating

[{'userId': 1000, 'movieId': 1081, 'rating': '5'},
 {'userId': 1000, 'movieId': 8580, 'rating': '5'},
 {'userId': 1000, 'movieId': 52975, 'rating': '4'},
 {'userId': 1000, 'movieId': 7614, 'rating': '5'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [85]:
new_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [89]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [91]:
# train a model using the new combined DataFrame
knn_basic_new = KNNBasic(sim_options={'name': 'pearson', 'user_based':True})
knn_basic_new.fit(new_data.build_full_trainset())

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [95]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
lst_movies = [(m_id, knn_basic_new.predict(1000, m_id)[3]) for m_id in new_df['movieId'].unique()]

In [97]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(lst_movies, key=lambda x:x[1])
ranked_movies

[(216, 1),
 (367, 1),
 (2450, 1),
 (2657, 1),
 (2826, 1),
 (344, 1),
 (160, 1),
 (2717, 1),
 (2072, 1),
 (1544, 1),
 (2411, 1),
 (2953, 1),
 (420, 1),
 (3040, 1),
 (3988, 1),
 (38, 1),
 (4040, 1),
 (122, 1),
 (1488, 1),
 (1976, 1),
 (2796, 1),
 (2906, 1),
 (4293, 1),
 (4396, 1),
 (4477, 1),
 (4686, 1),
 (2431, 1),
 (4215, 1),
 (2471, 1),
 (2122, 1),
 (5246, 1),
 (5247, 1),
 (4676, 1),
 (5248, 1),
 (2517, 1),
 (5428, 1),
 (1526, 1),
 (1996, 1),
 (4138, 1),
 (6315, 1),
 (4129, 1),
 (4204, 1),
 (4635, 1),
 (5771, 1),
 (5863, 1),
 (673, 1.5),
 (5, 1.5),
 (195, 1.5),
 (207, 1.5),
 (1917, 1.5),
 (3798, 1.5),
 (2412, 1.5),
 (2416, 1.5),
 (2423, 1.5),
 (2642, 1.5),
 (2719, 1.5),
 (3087, 1.5),
 (3394, 1.5),
 (3835, 1.5),
 (2021, 1.5),
 (4509, 1.5),
 (1642, 1.5),
 (2265, 1.5),
 (932, 1.5),
 (1515, 1.5),
 (4677, 1.5),
 (2103, 1.5),
 (4130, 1.5),
 (4799, 1.5),
 (2050, 1.5),
 (1389, 1.5),
 (3525, 1.5),
 (7026, 1.5),
 (1977, 1.5),
 (4081, 1.5),
 (6116, 1.5),
 (4526, 1.5),
 (3715, 1.5),
 (5213, 1.5),

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [98]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for i, mov_num in enumerate(user_ratings):
        title = movie_title_df.loc[movie_title_df['movieId'] == int(mov_num[0])]['title']
        print(f'Recommendation # {i + 1}: {title} \n')
        n -= 1
        if n == 0:
            break
            
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation # 1: 184    Billy Madison (1995)
Name: title, dtype: object 

Recommendation # 2: 325    Mask, The (1994)
Name: title, dtype: object 

Recommendation # 3: 1842    Howard the Duck (1986)
Name: title, dtype: object 

Recommendation # 4: 1997    Rocky Horror Picture Show, The (1975)
Name: title, dtype: object 

Recommendation # 5: 2126    13th Warrior, The (1999)
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.